# Milestone 3

In [ ]:
# Imports
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Read data
voter_df = pd.read_csv('../data/MA_l2_2022stats_2020block/MA_l2_2022stats_2020block.csv', low_memory=False).set_index('geoid20')
voter_df.shape

In [ ]:
# Group demographic columns together
registered = ['total_reg']
age = ['age_18_19', 'age_20_24', 'age_25_29','age_30_34','age_35_44', 'age_45_54', 'age_55_64', 'age_65_74','age_75_84', 'age_85over']
gender = [ 'voters_gender_m', 'voters_gender_f', 'voters_gender_unknown'] 
party = ['party_npp', 'party_dem', 'party_rep','party_lib', 'party_grn', 'party_con', 'party_ain', 'party_scl','party_oth', 'party_unk']
ethnicity1 = ['eth1_eur', 'eth1_hisp', 'eth1_aa',
                'eth1_esa', 'eth1_oth', 'eth1_unk']
ethnicity2 = ['eth2_euro', 'eth2_64',
                'eth2_93', 'eth2_10', 'eth2_30', 'eth2_23', 'eth2_66', 'eth2_34',
                'eth2_21', 'eth2_35', 'eth2_14', 'eth2_12', 'eth2_55', 'eth2_13',
                'eth2_32', 'eth2_61', 'eth2_85', 'eth2_29', 'eth2_33', 'eth2_38',
                'eth2_15', 'eth2_57', 'eth2_19', 'eth2_26', 'eth2_59', 'eth2_unk']
languages = ['languages_description_english', 'languages_description_spanish',
                'languages_description_portuguese',
                'languages_description_chinese', 'languages_description_italian',
                'languages_description_vietnamese', 'languages_description_other',
                'languages_description_unknown']
income = ['commercialdata_estimatedhhincomeamount_avg']

# Group variables related to the 2020 presidential election
presidential_2020 = ['g20201103_voted_all', 'g20201103_reg_all', 'g20201103_pct_voted_all',
                    'g20201103_voted_gender_m', 'g20201103_reg_gender_m',
                    'g20201103_pct_voted_gender_m', 'g20201103_voted_gender_f',
                    'g20201103_reg_gender_f', 'g20201103_pct_voted_gender_f',
                    'g20201103_voted_gender_unk', 'g20201103_reg_gender_unk',
                    'g20201103_pct_voted_gender_unk', 'g20201103_voted_eur',
                    'g20201103_reg_eur', 'g20201103_pct_voted_eur', 'g20201103_voted_hisp',
                    'g20201103_reg_hisp', 'g20201103_pct_voted_hisp', 'g20201103_voted_aa',
                    'g20201103_reg_aa', 'g20201103_pct_voted_aa', 'g20201103_voted_esa',
                    'g20201103_reg_esa', 'g20201103_pct_voted_esa', 'g20201103_voted_oth',
                    'g20201103_reg_oth', 'g20201103_pct_voted_oth', 'g20201103_voted_unk',
                    'g20201103_reg_unk', 'g20201103_pct_voted_unk']

## Data Cleaning and Processing

### Missingness

Issues to resolve:

- 'NO BLOCK ASSIGNMENT' entries (ignore or disaggregate to Census Blocks?)
- Missing income data (use mean/median income from Census Block Groups/Tracts?)

In [ ]:
# Code here

## Imbalance
Issues to resolve:
- Dominant party and ethnicity categories (?)
- Uneven distribution of voters among Census Blocks (use percentage values instead of counts in analysis OR use Census Block Groups?)

In [ ]:
# Code here